In [1]:
%autosave 180
%load_ext autoreload
%autoreload 2

# import matplotlib
# #matplotlib.use('qtagg')
%matplotlib tk
# import matplotlib.pyplot as plt
# plt.ion()
# %matplotlib notebook

#|
import os
import numpy as np
import time
import nidaqmx
from nidaqmx.constants import (AcquisitionType)  # https://nidaqmx-python.readthedocs.io/en/latest/constants.html
import numpy as np
import matplotlib.pyplot as plt
from nidaqmx.constants import TerminalConfiguration
import math
from multiprocessing import Process

# 



Autosaving every 180 seconds


In [4]:
##########################################################
########## COMPUTE VELOCITY FROM ROTARY ENCODER ##########
##########################################################
from tqdm import trange

#
def binarize(data):
    
    idx = np.where(data<2.5)[0]
    idx2 = np.where(data>=2.5)[0]
    
    data[idx] = 0
    data[idx2] = 1
    
    #
    return data

#
def get_velocity(rot1, rot2):
    
    # distance
    n_clicks_per_rotation = 500
    ball_diameter = 0.2  # distance in meters
    ball_circumference = ball_diameter*3.141592
    dist_per_click = ball_circumference/n_clicks_per_rotation
    
    # time
    sample_rate = 1000
    seconds_per_time_stamp = 1/sample_rate
    
    
    #
    bin1 = binarize(rot1)
    bin2 = binarize(rot2)
    
    #
    clicks = np.array((bin1, bin2)).T.squeeze()
    print (clicks)
        
    #
    vel = []
    times = []
    last_click_location = 0
    time_since_last_click = 0
    rot1_last_state = clicks[0,0]
    rot2_state = clicks[0,1]
    for k in trange(0,clicks.shape[0],1):
        
        #
        if clicks[k,0]!=rot1_last_state:
            distance = dist_per_click  #only walked 1 click
            time = time_since_last_click*seconds_per_time_stamp
            
            # 
            v = distance/time 
            
            #
            vel.append(v)
            times.append(k/sample_rate)       
            
            #
            time_since_last_click=0
            last_click_location=k #.copy()
            rot1_last_state=clicks[k,0]

        #    
        time_since_last_click+=1
        
    #
    return vel, times
    

##################################################################
##################################################################
##################################################################
data = np.load('/media/cat/4TB/donato/andres/DON-011733/20230127/data/results.npz',
               allow_pickle=True)

#
lick_detector_times = data['lick_detector_abstime']
print ("lick detector: ", lick_detector_times.shape, lick_detector_times)

#
reward_times = data['reward_times'].squeeze().T[:,1]
#print ("reward times: ", reward_times[:50])
ttl_times = data['ttl_times']
#print ("ttl times: ", ttl_times)

#
rot1 = data['rotary_encoder1_abstime']
rot2 = data['rotary_encoder2_abstime']
#print ("rotary 1: ", rot1)
#print ("rotary 2: ", rot2)

#
vel, times = get_velocity(rot1,rot2)

plt.figure()
plt.plot(times, vel)
plt.show()

lick detector:  (2675350, 1) [[-0.00226723]
 [-0.00210529]
 [-0.00210529]
 ...
 [-0.00194335]
 [-0.00226723]
 [-0.00210529]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


100%|█████████████████████████████████████████████████████████████████████████████████| 2675350/2675350 [00:00<00:00, 6488087.23it/s]
